In [10]:
import time
import os
import io

from random import seed
from random import randint

from argparse import ArgumentParser, Namespace
import yaml
from multiprocessing import cpu_count

import matplotlib.pyplot as plt

# default is to maximize the objectives
import time as time
import copy
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math
from scipy.stats import norm

# example of a gaussian process surrogate function
from math import sin
from math import pi
import numpy as np
from numpy import arange
from numpy import asarray
from numpy.random import normal
from numpy.random import uniform
from numpy.random import random
from numpy import cov
from numpy import mean
from numpy import std

from warnings import catch_warnings
from warnings import simplefilter


from autooed.utils.sampling import lhs
import random
#import xgboost as xgb
#from xgboost import XGBRegressor
#from xgboost import plot_tree
from sklearn import linear_model
from sklearn import ensemble
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import export_graphviz
from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.inspection import partial_dependence, plot_partial_dependence
from sklearn.decomposition import PCA
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# loocv to manually evaluate the performance of a random forest classifier
from sklearn.model_selection import LeaveOneOut, cross_val_score

from scipy.stats import pearsonr as pearsonr
from scipy import ndimage, misc
import pickle
import re
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from chainer_chemistry.datasets.molnet import get_molnet_dataset
# the package is in the same directory
# get Today's date from python!
from datetime import datetime
from autooed.utils.sampling import lhs
from autooed.problem import build_problem
from autooed.mobo import build_algorithm
from autooed.utils.seed import set_seed
from autooed.utils.initialization import generate_random_initial_samples, load_provided_initial_samples
from autooed.utils.plot import plot_performance_space, plot_performance_metric
from autooed.utils.plot import plot_performance_space_diffcolor
from argparse import ArgumentParser, Namespace
from arguments import get_args

import itertools

In [11]:
#### preprocessing 
# printability as Y
df = pd.read_csv('Yuchao_20220726.csv')

Printability = np.asarray (df['Printability']).reshape(1,-1).T
Tg = np.asarray (df['Tg']).reshape(1,-1).T
Tg[np.isnan(Tg)] = 200
Tg_group = [1 if 10<i<60 else 0 for i in Tg]
Tg_group = np.array(Tg_group)

toughness = np.asarray (df['Toughness(MJ/m3)']).reshape(1,-1).T
toughness[np.isnan(toughness)] = 0
strength = np.asarray (df['Tensile_Strength(MPa)']).reshape(1,-1).T
strength[np.isnan(strength)] = 0
strain = np.asarray (df['Tensile_Strain_percentage']).reshape(1,-1).T
strain[np.isnan(strain)] = 0

Y0 = Printability
Y = np.where(Y0 == 'Y', 1, 0)

#X_ = df.to_numpy()
A_Ratio = np.asarray (df['R1(HA)']).reshape(1,-1)
B_Ratio = np.asarray (df['R2(IA)']).reshape(1,-1)
C_Ratio = np.asarray (df['R3(NVP)']).reshape(1,-1)
D_Ratio = np.asarray (df['R4(AA)']).reshape(1,-1)
E_Ratio = np.asarray (df['R5(HEAA)']).reshape(1,-1)
F_Ratio = np.asarray (df['R6(IBOA)']).reshape(1,-1)

# did not consider F_Ratio, since we do not have it in optimization
X_ = np.concatenate((A_Ratio.T, B_Ratio.T, C_Ratio.T, D_Ratio.T, E_Ratio.T), axis=1)


# load monomers descriptors
df = pd.read_csv('monomers_info.csv')
energy = np.array (-df['dft_sp_E_RB3LYP'])
pol_area = np.array (df['polar_surface_area'])
complexity = np.array (df['complexity'])
HA = np.array (df['HA'])
solubility = np.array (df['solubility_sqrt_MJperm3'])
solubility_d = np.array (df['solubility_dipole'])
solubility_h = np.array (df['solubility_h'])
solubility_p = np.array (df['solubility_p'])

X0 = np.concatenate((A_Ratio.T, B_Ratio.T, C_Ratio.T, D_Ratio.T, E_Ratio.T, F_Ratio.T), axis=1)
X_energy = np.multiply (X0, energy)
#X_pol_area = np.multiply (X0, pol_area)
X_complexity = np.multiply (X0, complexity)
X_HA = np.multiply (X0, HA)
X_solubility_d = np.multiply (X0, solubility_d)
X_solubility_h = np.multiply (X0, solubility_h)
X_solubility_p = np.multiply (X0, solubility_p)

X = np.concatenate ((X_energy, X_complexity, X_HA, 
                    X_solubility_d, X_solubility_h, X_solubility_p), axis=1)

In [3]:
# find the best hyperparameters for RF printability
# best: max_depth = 5, n_estimators = 50
max_accuracy = 0
min_std = 1
parameter_pool = pd.DataFrame()
max_depths = [4, 5, 6, 8, 10]
#min_samples_splits = [2, 3]
n_estimatorsS = [50, 100]

best_parameters = pd.DataFrame()
best_parameters['max_depth'] = []
best_parameters['n_estimators'] = []

cv = LeaveOneOut()
for parameters in itertools.product(max_depths, n_estimatorsS):
    accuracy = list()
    accuracy_std = list()
    max_depth, n_estimators = parameters[0], parameters[1]
    print (max_depth, n_estimators)
    for i in range (10):
        RF_print = RandomForestClassifier(random_state=10*i, 
                                          max_depth = max_depth, 
                                          n_estimators = n_estimators)
                                          #min_samples_split=min_samples_split)
        scores = cross_val_score(RF_print, X, Tg_group, scoring='accuracy', cv=cv, n_jobs=-1)
        #print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))
        accuracy.append(mean(scores))
        accuracy_std.append(std(scores))
    if mean(accuracy)>max_accuracy:
        print ('better accuracy found for:', max_depth, n_estimators)
        max_accuracy = mean(accuracy)
        min_std = mean(accuracy_std)
        best_parameters['max_depth'] = [max_depth] 
        best_parameters['n_estimators'] = [n_estimators]
    elif mean(accuracy) == max_accuracy:
        if mean(accuracy_std) < min_std:
            print ('better accuracy found for:', max_depth, n_estimators)
            max_accuracy = mean(accuracy)
            min_std = mean(accuracy_std)
            best_parameters['max_depth'] = [max_depth] 
            best_parameters['n_estimators'] = [n_estimators]
            print('Accuracy: %.3f (%.3f)' % (mean(accuracy), std(accuracy_std)))

4 50
better accuracy found for: 4 50
4 100
5 50
better accuracy found for: 5 50
5 100
6 50
6 100
8 50
8 100
10 50
10 100


In [4]:
RF_print = RandomForestClassifier(random_state=10*i, 
                                  max_depth = best_parameters['max_depth'][0], 
                                  n_estimators = best_parameters['n_estimators'][0])
RF_print.fit(X, Y)
Yhat = RF_print.predict(X)
acc = accuracy_score(Y, Yhat)
print('Accuracy: %.3f' % acc)
print (RF_print.get_params(deep=True))

Accuracy: 1.000
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50, 'n_jobs': None, 'oob_score': False, 'random_state': 90, 'verbose': 0, 'warm_start': False}


<ipython-input-4-898841b278d9>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF_print.fit(X, Y)


In [5]:
# find the best hyperparameters for RF Tg
# best: max_depth = 5, n_estimators = 50
max_accuracy = 0
min_std = 1
parameter_pool = pd.DataFrame()
max_depths = [4, 5, 6, 8, 10]
#min_samples_splits = [2, 3]
n_estimatorsS = [50, 100]

best_parameters = pd.DataFrame()
best_parameters['max_depth'] = []
best_parameters['n_estimators'] = []

cv = LeaveOneOut()
for parameters in itertools.product(max_depths, n_estimatorsS):
    accuracy = list()
    accuracy_std = list()
    max_depth, n_estimators = parameters[0], parameters[1]
    print (max_depth, n_estimators)
    for i in range (10):
        RF_print = RandomForestClassifier(random_state=10*i, 
                                          max_depth = max_depth, 
                                          n_estimators = n_estimators)
                                          #min_samples_split=min_samples_split)
        scores = cross_val_score(RF_print, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
        #print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))
        accuracy.append(mean(scores))
        accuracy_std.append(std(scores))
    if mean(accuracy)>max_accuracy:
        print ('better accuracy found for:', max_depth, n_estimators)
        max_accuracy = mean(accuracy)
        min_std = mean(accuracy_std)
        best_parameters['max_depth'] = [max_depth] 
        best_parameters['n_estimators'] = [n_estimators]
    elif mean(accuracy) == max_accuracy:
        if mean(accuracy_std) < min_std:
            print ('better accuracy found for:', max_depth, n_estimators)
            max_accuracy = mean(accuracy)
            min_std = mean(accuracy_std)
            best_parameters['max_depth'] = [max_depth] 
            best_parameters['n_estimators'] = [n_estimators]
            print('Accuracy: %.3f (%.3f)' % (mean(accuracy), std(accuracy_std)))
print (max_accuracy, min_std)

4 50
better accuracy found for: 4 50
4 100
better accuracy found for: 4 100
5 50
better accuracy found for: 5 50
5 100
6 50
6 100
8 50
8 100
10 50
10 100
0.9170731707317075 0.27348813456895726


In [12]:
RF_Tg = RandomForestClassifier(random_state=10*i, 
                                  max_depth = best_parameters['max_depth'][0], 
                                  n_estimators = best_parameters['n_estimators'][0])
RF_Tg.fit(X, Tg_group)
Yhat = RF_Tg.predict(X)
acc = accuracy_score(Tg_group, Yhat)
print('Accuracy: %.3f' % acc)
print (RF_Tg.get_params(deep=True))

Accuracy: 1.000
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50, 'n_jobs': None, 'oob_score': False, 'random_state': 90, 'verbose': 0, 'warm_start': False}


In [13]:
RF_Tg.predict_proba(X)

array([[0.  , 1.  ],
       [0.06, 0.94],
       [0.24, 0.76],
       [0.26, 0.74],
       [0.04, 0.96],
       [0.  , 1.  ],
       [0.12, 0.88],
       [0.  , 1.  ],
       [0.8 , 0.2 ],
       [0.64, 0.36],
       [0.78, 0.22],
       [0.1 , 0.9 ],
       [0.76, 0.24],
       [0.1 , 0.9 ],
       [0.28, 0.72],
       [0.  , 1.  ],
       [0.2 , 0.8 ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.1 , 0.9 ],
       [0.06, 0.94],
       [0.02, 0.98],
       [0.04, 0.96],
       [0.02, 0.98],
       [0.08, 0.92],
       [0.02, 0.98],
       [0.94, 0.06],
       [0.98, 0.02],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [0.92, 0.08],
       [1.  , 0.  ],
       [0.  , 1.  ],
       [0.82, 0.18],
       [0.08, 0.92],
       [0.84, 0.16],
       [0.96, 0.04],
       [1.  , 0.  ],
       [0.18, 0.82],
       [0.94, 0.06],
       [0.  , 1.  ]])

In [9]:
Tg_group

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1])